# Body tracking software
This model will track a pose and normalize it relative to the middle of their collarbone, their torso, and its perpedicular, hopefully giving a rotationally, translationally, and scaling-ly normalized body

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
vid = cv2.VideoCapture(1)

In [ ]:
model = tf.lite.Interpreter(model_path="model.tflite")
model.allocate_tensors()

def get_details(image):
    resized = cv2.resize(image, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
    resized = np.expand_dims(resized, 0).astype("uint8")
    input_details = model.get_input_details()
    output_details = model.get_output_details()
    model.set_tensor(input_details[0]['index'], resized)
    model.invoke()
    keypoints = model.get_tensor(output_details[0]['index'])[0][0]

    returned_keypoints = [[image.shape[0] * i[0], image.shape[1] * i[1], i[2]] for i in keypoints]
    return returned_keypoints

In [ ]:
parts = ["nose", "left eye", "right eye", "left ear", "right ear", "left shoulder", "right shoulder", "left elbow", "right elbow", "left wrist", "right wrist", "left hip", "right hip", "left knee", "right knee", "left ankle", "right ankle"]
part_to_ind = {part: i for i, part in enumerate(parts)}

In [ ]:
while True:
    ret, frame = vid.read()

    frame = 255 - frame
    positions = np.array(get_details(frame))
    cv2.imshow("image", frame)

    plt.figure()
    plt.imshow(frame)
    
    plt.scatter([i[1] for i in positions], [i[0] for i in positions], s=200)
    plt.show()

    time.sleep(0.5)